In [1]:
import trimet_plan_game_reset as tpg
import geopandas as gpd

In [2]:
origin_str = "45.399445, -122.659634"
# "45.495953, -122.557314"
# "45.495953, -122.557314"
# "45.582637, -122.754038"
# "45.512908, -122.711987"
destination_str = "45.457311, -122.574757"
# "45.517341, -122.401294"
# "45.517341, -122.401294"
# "45.512894, -122.656701"
#  "45.510896, -122.611478"
planner_response = tpg.call_planner(origin_str, destination_str)

In [19]:
tm_boundary = gpd.read_file("tm_route_buffer_bounds.geojson")
trimet_crs = "EPSG:2913"
error_length = 5
tries = 1
while error_length > 0:
    gdf_points, dist_btw_points = tpg.generate_points_within_tm_boundary(tm_boundary, trimet_crs)
    fromplace = gdf_points[gdf_points['point_type']=='origin']['points_str'].to_numpy()[0]
    toplace = gdf_points[gdf_points['point_type']=='destination']['points_str'].to_numpy()[0]
    json_content = tpg.call_planner(fromplace, toplace)
    #if there's no error message key = 'error' will not be present so '' is returned
    #len('') is 0 so error_length becomes 0
    error_length = len(json_content.get('error',''))
    # print(error_length)
    if error_length == 0:
        #adding logic to make sure the trip planning is not crazy long
        #this is a crude way of making sure there is 1 itinerary with 2 routes
        itineraries_df = tpg.get_itinerary_paths(json_content)
        max_route_count = (itineraries_df
                            [itineraries_df['route_id']!='WALK']
                            .groupby(['itin_idx'])
                            .agg(route_count=('route_id','nunique')
                                            ,totalTime=('totalTime','max'))
                            .reset_index()
                            .sort_values('totalTime')
                            .head(3)
                            ['route_count'].max()
                            )

        if max_route_count > 2:
            error_length = 1
    tries += 1
print(f"number of tries {tries}")
itineraries_reduced, itinerary_routes_reduced = tpg.create_intinerary_gdf_and_reduce(itineraries_df)

number of tries 2


In [20]:
itineraries_reduced

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry,route_id_combo
0,0,5640,2078,2677,885,2258.556075,0,WALK,WALK,,Origin,10120,Washington Park MAX Station,"MULTILINESTRING ((-122.71540 45.52048, -122.71...",100 to 87
1,0,5640,2078,2677,885,2258.556075,1,100,TRAM,10120,Washington Park MAX Station,8347,Gateway/NE 99th Ave TC MAX Station,"LINESTRING (-122.71634 45.51062, -122.71593 45...",100 to 87
2,0,5640,2078,2677,885,2258.556075,3,87,BUS,10864,Gateway Transit Center,6528,NE 102nd & Shaver,"LINESTRING (-122.56312 45.52989, -122.56315 45...",100 to 87
3,1,4942,1828,2769,345,1880.270657,0,WALK,WALK,,Origin,10120,Washington Park MAX Station,"MULTILINESTRING ((-122.71540 45.52048, -122.71...",100 to 22
4,1,4942,1828,2769,345,1880.270657,1,100,TRAM,10120,Washington Park MAX Station,8347,Gateway/NE 99th Ave TC MAX Station,"LINESTRING (-122.71634 45.51062, -122.71593 45...",100 to 22
5,1,4942,1828,2769,345,1880.270657,3,22,BUS,10864,Gateway Transit Center,5228,NE Shaver & 109th,"LINESTRING (-122.56312 45.52989, -122.56315 45...",100 to 22
6,3,5644,1705,3681,258,1831.099269,0,WALK,WALK,,Origin,685,W Burnside & SW Fischer Ln,"MULTILINESTRING ((-122.71540 45.52048, -122.71...",20 to 73
7,3,5644,1705,3681,258,1831.099269,1,20,BUS,685,W Burnside & SW Fischer Ln,5425,SE Stark & 122nd,"LINESTRING (-122.72227 45.51924, -122.72217 45...",20 to 73
8,3,5644,1705,3681,258,1831.099269,3,73,BUS,6652,SE 122nd & Oak,4734,NE Prescott & 110th,"LINESTRING (-122.53779 45.51988, -122.53779 45...",20 to 73


In [12]:
len(planner_response.get('error',''))

0

In [9]:
itineraries_df = tpg.get_itinerary_paths(planner_response)

3

In [8]:
itineraries_reduced, itinerary_routes_reduced = tpg.create_intinerary_gdf_and_reduce(itineraries_df)

In [9]:
itineraries_reduced

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry,route_id_combo
0,0,3478,389,2171,918,512.015194,0,WALK,WALK,,Origin,4562,SE Powell & 102nd,"MULTILINESTRING ((-122.55730 45.49583, -122.55...",9 to 20
1,0,3478,389,2171,918,512.015194,1,9,BUS,4562,SE Powell & 102nd,12757,NE 10th & Hood (Gresham Central TC),"LINESTRING (-122.55799 45.49578, -122.55797 45...",9 to 20
2,0,3478,389,2171,918,512.015194,3,20,BUS,2253,Gresham Central Transit Center,10879,"NE Kane & 29th, MHCC","LINESTRING (-122.42647 45.50264, -122.42633 45...",9 to 20
3,1,3615,392,2330,893,503.706955,0,WALK,WALK,,Origin,4562,SE Powell & 102nd,"MULTILINESTRING ((-122.55730 45.49583, -122.55...",9 to 73 to 20
4,1,3615,392,2330,893,503.706955,1,9,BUS,4562,SE Powell & 102nd,4573,SE Powell & 122nd,"LINESTRING (-122.55799 45.49578, -122.55797 45...",9 to 73 to 20
5,1,3615,392,2330,893,503.706955,3,73,BUS,6655,SE 122nd & Powell,6652,SE 122nd & Oak,"LINESTRING (-122.53767 45.49740, -122.53767 45...",9 to 73 to 20
6,1,3615,392,2330,893,503.706955,5,20,BUS,5425,SE Stark & 122nd,5397,SE Stark & NE Hale,"LINESTRING (-122.53724 45.51902, -122.53699 45...",9 to 73 to 20


In [13]:
import datetime 
import requests
import json
import os
mapbox_api_key = os.getenv('mapbox_api_key','')

In [2]:
def call_mapbox(fromPlace, toPlace):
    ''' 
    fromPlace = "lat, lon"
    toPlace = "lat,lon"

    mapbox takes lon, lat
    '''
    date = datetime.datetime.now().strftime("%Y-%m-%d")
    base_url = "https://api.mapbox.com/directions/v5/mapbox/driving-traffic/"
    time="12:00"
    depart_time=f"{date}T{time}"
    from_lon = fromPlace.split(",")[-1]
    from_lat = fromPlace.split(",")[0]
    to_lon = toPlace.split(",")[-1]
    to_lat = toPlace.split(",")[0]
    base_url_w_origin_dest = f"{base_url}{from_lon},{from_lat};{to_lon},{to_lat}"
    r = requests.get(url=base_url_w_origin_dest, params={'depart_at':depart_time
                                    ,'access_token':mapbox_api_key})
    assert(r.status_code==200)
    return r.json()

In [3]:
origin_str = "45.495953, -122.557314"
destination_str = "45.517341, -122.401294"

In [5]:
mapbox_response = call_mapbox(origin_str, destination_str)
mapbox_response

{'routes': [{'weight_typical': 1218.102,
   'duration_typical': 1026.988,
   'weight_name': 'auto',
   'weight': 1218.102,
   'duration': 1026.988,
   'distance': 20983.676,
   'legs': [{'via_waypoints': [],
     'admins': [{'iso_3166_1_alpha3': 'USA', 'iso_3166_1': 'US'}],
     'weight_typical': 1218.102,
     'duration_typical': 1026.988,
     'weight': 1218.102,
     'duration': 1026.988,
     'steps': [],
     'distance': 20983.676,
     'summary': 'I 205 North, I 84 East'}],
   'geometry': '_|ttGb~_kVNfK_Hj`@oKXevDrAcxAmUmScL{EyIo@i}@`_@c|FOalCnR_uEsA_i@yLwdAbJgs@|Boc@fm@H`CoEn@}Q`EqFtp@XvR}C@weAvIB'}],
 'waypoints': [{'distance': 12.781,
   'name': 'Southeast Powell Boulevard',
   'location': [-122.557296, 45.495839]},
  {'distance': 9.161,
   'name': 'Northeast Hale Avenue',
   'location': [-122.401179, 45.517326]}],
 'code': 'Ok',
 'uuid': 'VEMsVScGZ4X44mPdV96sjA3X1LFv3iURYyjtk7G7as6bclWVCEoXtw=='}